<a href="https://colab.research.google.com/github/keran-w/speech-lexical-segmentation/blob/main/runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title upload
# from google.colab import files
# uploaded = files.upload()
# dataset_dir = next(iter(uploaded))
# !unzip {dataset_dir}
# dataset_dir = dataset_dir[:-4]

from google.colab import drive
drive.mount('/content/drive')
# dataset_dir = 'Lost_in_the_Jungle' # 95/97 (57)
# dataset_dir = 'B7-05_The_Willow_Pattern_Plot' # 99/99
# dataset_dir = 'B7-08_The_Joke_Machine' # 87/93
# dataset_dir = 'Australian_Adventure' # 98/98
dataset_dir = 'a_day_in_london'
# dataset_dir = 'the_rainbow_machine'
# !rm -r {dataset_dir}
!unzip drive/MyDrive/Datasets/Audio_Segmentation/{dataset_dir}.zip -d .

In [ ]:
# @title initialize
! pip install pydub
# ! pip install speechbrain
! pip install transformers
! pip install similarities

import re
import os
import IPython
import numpy as np

import re
from similarities import Similarity

import torch
import torchaudio

from pydub import AudioSegment
from time import time
import json

from tqdm import tqdm
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

import speechbrain as sb
from speechbrain.dataio.dataio import read_audio
import numpy as np
from IPython.display import Audio 
from more_itertools import consecutive_groups

# from speechbrain.pretrained import SepformerSeparation

import logging
logging.disable(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

torch.random.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# separator = SepformerSeparation.from_hparams(source="speechbrain/sepformer-wsj02mix", savedir='pretrained_models/sepformer-wsj02mix')
processor = Wav2Vec2Processor.from_pretrained('jonatasgrosman/wav2vec2-large-xlsr-53-english')
model = Wav2Vec2ForCTC.from_pretrained('jonatasgrosman/wav2vec2-large-xlsr-53-english').to(device)

def if_any_alpha(str):
    for c in str:
        if c.isalpha(): return False
    return True

idx_list = []
for p in os.listdir(f'{dataset_dir}/audio/'):
    segs = p.split('.')
    if segs[1] == 'mp3': 
        try: idx_list.append(int(segs[0]))
        except: pass
idx_list = sorted(idx_list)

In [ ]:
# @title run
def process(audio_path, json_path):
    waveform, sample_rate = torchaudio.load(audio_path)
    waveform = torchaudio.functional.resample(waveform[1].to(device), sample_rate, 16000)

    input_values = processor(waveform, return_tensors='pt', sampling_rate=16000, padding='longest').input_values.to(device)
    logits = model(input_values).logits.squeeze()

    fragments = json.load(open(json_path, 'r'))['fragments']
    texts = [d['lines'][0].lower() for d in fragments]
    decodings = processor.decode(logits.argmax(1), output_char_offsets=True).char_offsets
    flag = word_start_offset = prev_word_end_offset = 0
    word = ''
    res = []

    for decoding in decodings:
        char, start_offset, end_offset = decoding.values()
        if start_offset > end_offset: start_offset, end_offset = end_offset, start_offset

        if char not in (' ', '-'):
            word += char
            if word_start_offset == 0: word_start_offset = start_offset
            prev_word_end_offset = end_offset
        else:
            res.append([word, word_start_offset, prev_word_end_offset, prev_word_end_offset - word_start_offset])
            word = ''
            word_start_offset = 0

    res = np.array(res)
    filter_texts = list(filter(None, [re.sub('[^a-z]+', '', text) for text in texts]))
    filter_res = [re.sub('[^a-z]+', '', text) for text in res[:, 0]]

    if len(filter_texts) != len(filter_res):

        matched_list = []
        min_j = 0
        for i, word_res in enumerate(filter_res):
            for j, word_text in enumerate(filter_texts):
                if j < min_j:
                    continue
                if word_res == word_text:
                    min_j = j
                    matched_list.append([i, j])
                    break

        matched_list = np.array(matched_list)
        res_lst, texts_lst = np.setdiff1d(np.arange(len(filter_res)), matched_list[:, 0]), np.setdiff1d(np.arange(len(filter_texts)), matched_list[:, 1])

        if len(filter_texts) < len(filter_res):

            similarity = Similarity(model_name_or_path='bert-base-uncased', similarity_type='cosine', embedding_type='sbert')
            corpus = [filter_texts[ti] for ti in texts_lst]
            similarity.add_corpus(corpus)

            sentences = res[res_lst, 0]
            sentences_list = [(*sentences[:si], sentences[si] + sentences[si+1], *sentences[si+2:]) for si in range(len(sentences) - 1)]

            max_sim_sum, sen = 0, None
            for sentences in sentences_list:
                sim = similarity.most_similar(queries=list(sentences), topn=1)
                sim_sum = sum([list(sim_val.values())[0] for sim_val in sim.values()])
                if sim_sum > max_sim_sum:
                    max_sim_sum, sen = sim_sum, sentences

            for ri, _ in enumerate(res[:-1]):
                if res[ri, 0] == '': continue
                if res[ri, 0] + res[ri + 1, 0] in sen:
                    res[ri] = res[ri, 0] + res[ri + 1, 0], res[ri, 1], res[ri + 1, 2], ''
                    res[ri + 1] = '', '', '', ''

            res = res[res[:, 0] != '']

        elif len(filter_texts) > len(filter_res):
            
            add_rows = []
            for rl, tl in zip([list(group) for group in consecutive_groups(res_lst)], [list(group) for group in consecutive_groups(texts_lst)]):
                if len(rl) == len(tl) or len(rl) > 1: continue
                rl = rl[0]
                start, end = int(res[rl][1]), int(res[rl][2])
                l = logits[start:end+1]
                l_idx = l[:, 4].argmax().item()
                add_text = processor.decode(l[l_idx:].argmax(1))
                if add_text != '':
                    add_rows.append((rl, [add_text, start + l_idx + 1, end, end - (start + l_idx + 1)]))

            count = 1
            for ari, ar in add_rows:
                res = np.insert(res, ari + count, ar, 0)
                count += 1
        
        filter_res = [re.sub('[^a-z]+', '', text) for text in res[:, 0]]
    return decodings, res, filter_texts, filter_res

f = open(f'{dataset_dir}.txt', 'w')
correct_info = []
wrong_info = []
wrong_idx_list = []
for idx in tqdm(idx_list):
    audio_path = f'{dataset_dir}/audio/{idx}.mp3'
    json_path = f'{dataset_dir}/json/G-({idx}).json'
    try: open(json_path, 'r')
    except: 
        f.write(f'File Not Found: {json_path}\n')
        continue

    decodings, res, filter_texts, filter_res = process(audio_path, json_path)
    duration = AudioSegment.from_file(audio_path).duration_seconds
    scale = decodings[-1]['end_offset'] / duration

    markers = json.load(open(json_path, 'r'))
    r = [int(res_row[1]) for res_row in res] + [decodings[-1]['end_offset']]
    s = [(r[ri], r[ri+1]) for ri in range(len(r) - 1)]
        
    for (b, e), marker in zip(s, markers['fragments']):
        marker['begin'] = f'{int(b) / scale:.3f}'
        marker['end'] = f'{int(e) / scale:.3f}'

    f.write(f'\nG-{idx}.json\n')
    f.write(json.dumps(markers, indent = 4))

    if len(filter_texts) != len(filter_res):
        wrong_idx_list.append(idx)
        wrong_info.append(f"{idx:2d}.mp3 - original({len(filter_texts)}): {' ' if len(filter_texts) < 10 else ''}     {' '.join(filter_texts)}")
        wrong_info.append(f"{idx:2d}.mp3 - prediction({len(filter_res)}): {' ' if len(filter_res) < 10 else ''}   {' '.join(filter_res)}")

    else:
        correct_info.append(f"{idx:2d}.mp3 - original:          {' '.join(filter_texts)}")
        correct_info.append(f"{idx:2d}.mp3 - prediction:        {' '.join(filter_res)}")

f.write('\ncorrect_info\n')
f.write('\n'.join(correct_info))
f.write('\nwrong_info\n')
f.write('\n'.join(wrong_info))
f.close()
from google.colab import files
files.download(f'{dataset_dir}.txt')